<a href="https://colab.research.google.com/github/rohithsing/SAP/blob/main/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 1. Imports
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [2]:
# 2. Load Dataset
VOCAB_SIZE = 10000   # top 10k frequent words
MAX_LEN = 200        # max words per review

(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=VOCAB_SIZE)

# Padding for equal sequence length
X_train = pad_sequences(X_train, maxlen=MAX_LEN)
X_test = pad_sequences(X_test, maxlen=MAX_LEN)

17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [3]:
# 3. Build LSTM Model
model = Sequential([
    Embedding(VOCAB_SIZE, 128, input_shape=(MAX_LEN,)),
    LSTM(128),
    Dense(1, activation="sigmoid")
])

model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:100: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 200, 128)       │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 128)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,411,713 (5.39 MB)

 Trainable params: 1,411,713 (5.39 MB)

 Non-trainable params: 0 (0.00 B)

In [4]:
# 4. Compile Model
model.compile(
    loss="binary_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

In [5]:
# 5. Train Model
model.fit(
    X_train,
    y_train,
    epochs=5,
    batch_size=64,
    validation_split=0.2
)

Epoch 1/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 8s 14ms/step - accuracy: 0.7067 - loss: 0.5405 - val_accuracy: 0.7344 - val_loss: 0.5381
Epoch 2/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 5s 17ms/step - accuracy: 0.8647 - loss: 0.3304 - val_accuracy: 0.8612 - val_loss: 0.3383
Epoch 3/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.9325 - loss: 0.1835 - val_accuracy: 0.8580 - val_loss: 0.3568
Epoch 4/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - accuracy: 0.9563 - loss: 0.1229 - val_accuracy: 0.8314 - val_loss: 0.3836
Epoch 5/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step - accuracy: 0.9550 - loss: 0.1287 - val_accuracy: 0.8538 - val_loss: 0.4519


In [6]:
#6. Evaluate Model
test_loss, test_accuracy = model.evaluate(X_test, y_test)

print("\nTest Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

782/782 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.8447 - loss: 0.4683

Test Loss: 0.4604267477989197
Test Accuracy: 0.8471999764442444


In [7]:
# 7. Correct Review Encoding
word_index = imdb.get_word_index()

def encode_review(text):
    """
    Encode text the same way IMDB was trained
    """
    encoded = [1]  # start token

    for word in text.lower().split():
        idx = word_index.get(word)
        if idx is not None and idx < VOCAB_SIZE:
            encoded.append(idx + 3)
        else:
            encoded.append(2)  # unknown word

    return pad_sequences([encoded], maxlen=MAX_LEN)

1641221/1641221 ━━━━━━━━━━━━━━━━━━━━ 1s 1us/step


In [8]:
# 8. Predictions
reviews = [
    "this movie was good and enjoyable",
    "this movie was bad and boring",
    "i loved this film it was amazing",
    "i hated this movie it was terrible"
]

for review in reviews:
    score = model.predict(encode_review(review))[0][0]
    print("\nReview:", review)
    print("Sentiment Score:", score)
    print("Prediction:", "Positive 😊" if score > 0.5 else "Negative 😞")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step

Review: this movie was good and enjoyable
Sentiment Score: 0.77157915
Prediction: Positive 😊
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step

Review: this movie was bad and boring
Sentiment Score: 0.06847036
Prediction: Negative 😞
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step

Review: i loved this film it was amazing
Sentiment Score: 0.982206
Prediction: Positive 😊
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step

Review: i hated this movie it was terrible
Sentiment Score: 0.051348086
Prediction: Negative 😞
